### Task 1.1

Shit....! Here we go again!

We are looking at Private individuals vs Business actors again

In [1]:
import numpy as np
import pandas as pd
from image_analysis.data import get_images_dataframe

from sdm.config import get_db_connection

In [2]:
conn = get_db_connection(db_type="sqlite", db_path="D:/UU/Sem3/SDM/social_data_mining/data/twitter.db")
df = get_images_dataframe(db=conn)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38980 entries, 0 to 38979
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   account_id    38980 non-null  object
 1   image_id      38980 non-null  object
 2   account_type  38980 non-null  object
 3   lang          38729 non-null  object
 4   stance        38441 non-null  object
dtypes: object(5)
memory usage: 1.5+ MB
None


,account_id,image_id,account_type,lang,stance
0,8508262,nan,Private individuals,fr,For
1,8508262,nan,Private individuals,fr,For
2,8508262,nan,Private individuals,fr,For
3,8508262,nan,Private individuals,fr,For
4,8508262,nan,Private individuals,fr,For


In [6]:
valid_img_df = df.copy()
valid_img_df["image_id"] = valid_img_df["image_id"].apply(lambda x: np.nan if x == "nan" else x)
valid_img_df = valid_img_df.dropna(subset=["image_id"])
# valid_img_df["image_id"] = valid_img_df["image_id"].apply(lambda x: x["media_keys"])
valid_img_df['media_keys'] = valid_img_df['image_id'].apply(
    lambda x: x['media_keys'] if isinstance(x, dict) and 'media_keys' in x else None
)
valid_img_df = valid_img_df.reset_index(drop=True)
print(valid_img_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16387 entries, 0 to 16386
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   account_id    16387 non-null  object
 1   image_id      16387 non-null  object
 2   account_type  16387 non-null  object
 3   lang          16260 non-null  object
 4   stance        16188 non-null  object
 5   media_keys    0 non-null      object
dtypes: object(6)
memory usage: 768.3+ KB
None


In [4]:
valid_img_df.head()

,account_id,image_id,account_type,lang,stance
0,8508262,{'media_keys': ['3_669872845984546817']},Private individuals,fr,For
1,8508262,{'media_keys': ['3_671208626439213056']},Private individuals,fr,For
2,8508262,{'media_keys': ['3_671223094661545984']},Private individuals,fr,For
3,8508262,{'media_keys': ['3_671231051176517632']},Private individuals,fr,For
4,8508262,{'media_keys': ['3_671231164783398913']},Private individuals,fr,For
